In [1]:
from utils.tobj import import_tobj
import meshio
import numpy as np


model = "boatv3"
V, T = import_tobj(f"assets/{model}/{model}.tobj")
mesh = meshio.Mesh(points = V, cells = [("tetra", T)])
mesh.write(f"assets/{model}.msh", file_format="gmsh22")

(18395, 3) (58768, 4)


Warning: Appending zeros to replace the missing physical tag data.

Warning: Appending zeros to replace the missing geometrical tag data.

In [3]:
obj_in = meshio.read("assets/bunny.obj")
off_out = meshio.write("assets/bunny.off", obj_in, file_format="off")

In [3]:
V.shape, T.shape

AttributeError: 'NoneType' object has no attribute 'shape'